# Stats about Train / Dev / Test sets

In [1]:
import pandas as pd

import sys
sys.path.insert(0, '../')
from utils.config import PATHS
from utils.latex import show_latex, TABLES

In [2]:
domains=['ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']

# Domains classifier

## Load data

In [3]:
datapath = PATHS.getpath('data_expr_sept')
train = pd.read_pickle(datapath / 'clf_domains/train_excl_bck_add_pilot.pkl')
test = pd.read_pickle(datapath / 'clf_domains/test.pkl')
dev = pd.read_pickle(datapath / 'clf_domains/dev.pkl')

## Check correct split

i.e. there are no notes that appear in more than one set

In [4]:
print(test.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(test.NotitieID).any())

False
False
False


## Number of sentences

- A sentence can contain more than one domain and therefore be counted more than once.
- The last column is the total number of sentences in the dataset (incl. all negative examples)

In [5]:
caption = "Domain classification: datasets, sentence-level"
label = "domains_datasets_sents"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'pad_sen_id']]),
    columns = domains,
    data = data.labels.to_list()
)

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_sentences')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_sentences
dataset,,,,,,,,,,
dev,411,22,29,105,225,119,127,96,147,"21,742"
test,775,39,54,160,382,253,287,125,181,"22,082"
train,"4,988",247,486,989,"2,420","2,489","1,967",755,"3,390","239,153"
total,"6,174",308,569,"1,254","3,027","2,861","2,381",976,"3,718","282,977"


## Number of notes

In [6]:
caption = "Domain classification: datasets, note-level"
label = "domains_datasets_notes"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'NotitieID']]),
    columns = domains,
    data = data.labels.to_list()
).groupby(level=[0,1]).any()

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_notes')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_notes
dataset,,,,,,,,,,
dev,188,17,25,71,128,75,78,71,83,431
test,231,27,34,92,165,95,116,64,94,431
train,"2,345",175,381,707,"1,416","1,631","1,260",546,"1,989","6,821"
total,"2,764",219,440,870,"1,709","1,801","1,454",681,"2,166","7,683"


# Levels classifiers

## Number of sentences

In [7]:
caption = "Levels classification: datasets, sentence-level"
label = "levels_datasets_sents"

table = pd.DataFrame(index=['train', 'dev', 'test'])

for dom in domains:
    datapath = PATHS.getpath('data_expr_sept') / f'clf_levels_{dom}_sents'
    train = pd.read_pickle(datapath / 'train.pkl')
    test = pd.read_pickle(datapath / 'test.pkl')
    dev = pd.read_pickle(datapath / 'dev.pkl')
    table.loc['train', dom] = len(train)
    table.loc['dev', dom] = len(dev)
    table.loc['test', dom] = len(test)

table.astype(int).pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
train,"5,233",251,216,"1,005","2,491","1,086","1,104",766,"1,420"
dev,440,23,29,107,236,124,132,98,148
test,421,32,26,100,183,139,136,60,155


## Number of notes

In [8]:
caption = "Levels classification: datasets, note-level"
label = "levels_datasets_notes"

table = pd.DataFrame(index=['train', 'dev', 'test'])

for dom in domains:
    datapath = PATHS.getpath('data_expr_sept') / f'clf_levels_{dom}_sents'
    train = pd.read_pickle(datapath / 'train.pkl')
    test = pd.read_pickle(datapath / 'test.pkl')
    dev = pd.read_pickle(datapath / 'dev.pkl')
    table.loc['train', dom] = train.NotitieID.nunique()
    table.loc['dev', dom] = dev.NotitieID.nunique()
    table.loc['test', dom] = test.NotitieID.nunique()

table.astype(int).pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM
train,"2,344",175,162,705,"1,417",717,699,536,792
dev,189,17,25,71,128,74,77,71,83
test,200,21,22,70,123,79,74,41,84


# Save tables

In [9]:
prefix = 'ml_datasets'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)